# ニワゼキショウ： 遺伝的多様性解析2

## 背景
- ニワゼキショウ
  - 一年生植物。まれに多年生
  - 四倍体
  - 種内に二つの花色（白花と紫花）がある
- 日本のニワゼキショウ集団のあいだで分化程度があるのか、花色のあいだで分化があるのかはよくわかっていない。

## 目的
- ニワゼキショウのサンプル間や花色間の分化程度を明らかにする

## 調べること
- サンプル間はどの程度遺伝的に分化しているのか？
- 白花と紫花の間は遺伝的に分化しているのか？
- サンプリング地点間で地理的距離があるほど、遺伝的に分化しているのか？

## サンプル

サンプル情報: 
- CEL_W: 白花10個体@栽培植物起原学研究室
- FUK_W: 白花4個体@福西公園
- NIS_W: 白花10個体@西竹の里町
- KAT_W: 白花20個体@桂キャンパス
- KUR_P: 紫花5個体@物集女車塚古墳
- FUK_P: 紫花9個体@福西公園
- ONO_P: 紫花10個体@大原野灰方町
- KAT_P: 紫花20個体@桂キャンパス

複数個体の花のつぼみをプールし、1サンプルとした。

サンプリング地点:  
https://www.google.com/maps/d/u/0/edit?mid=1vQLtjaIn89F35jzfvA0PuKAxKQ6cbfkx&usp=sharing

## 使用データ
- SNPデータセット
https://raw.githubusercontent.com/qqep685d/ilas2021_niwazekisho/master/SNP_dataset.txt



In [ ]:
### データをダウンロード
!wget -O 'SNP_dataset.txt' https://raw.githubusercontent.com/qqep685d/ilas2021_niwazekisho/master/SNP_dataset.txt

## データを読み込む

In [ ]:
import pandas as pd
df = pd.read_csv("SNP_dataset.txt", sep="\t", header=0)
df

## 使用する遺伝子数とSNP数

In [ ]:
# 遺伝子数
df.groupby('Gene_ID').count().shape[0]

In [ ]:
# 遺伝子数
df.shape[0]

## 準備: 各サンプルの遺伝的多様性（ヘテロ接合度）を得る

　サンプル間の遺伝的分化（Fst）は次式で得ることができる。

Fst = 1 - (H_1 + H_2)/2 / H_t
- H_t: 統合サンプルのヘテロ接合度
- H_1: サンプル1のヘテロ接合度
- H_2: サンプル2のヘテロ接合度

　したがって、Fstを得るためには、それぞれのヘテロ接合度を求める必要がある。

ヘテロ接合度の求め方:  
- A1の割合 = (A1アリルの個数) / ((A1アリルの個数) + (A2アリルの個数))
- A2の割合 = 1 - (A1の割合)
- ヘテロ接合度 = 2 * (A1の割合) * (A2の割合)


In [ ]:
### 各サンプルのヘテロ接合度を準備する

### サンプル名
samples = ["CEL_W", "FUK_W", "NIS_W", "KAT_W", "KUR_P", "FUK_P", "ONO_P", "KAT_P"]
for s in samples:
  ## アリル頻度
  freqA1 = df[s+"_A1"]/(df[s+"_A1"] + df[s+"_A2"])
  freqA2 = 1 - freqA1
  ## ヘテロ接合度
  df[s+"_het"] = 2 * freqA1 * freqA2

### 各サンプルのヘテロ接合度データ
one = df.iloc[:, -8:].mean()
one

## 二つのサンプルを合わせた「統合サンプル」のヘテロ接合度を得る

（例） CEL_WとFUK_Wの統合サンプルを考える

- 統合サンプルに含まれるアリルの個数
  - 合計アリル数 = CEL_W_A1 + CEL_W_A2 + FUK_W_A1 + FUK_W_A2
  - A1アリル数 = CEL_W_A1 + FUK_W_A1
  - A2アリル数 = CEL_W_A2 + FUK_W_A2

- 統合サンプルのアリル頻度
  - A1アリル頻度 = A1アリル数 / 合計アリル数
  - A2アリル頻度 = A2アリル数 / 合計アリル数 = 1 - A1アリル頻度

- 統合サンプルのヘテロ接合度
 - ヘテロ接合度 = 2 * A1アリル頻度 * A2アリル頻度

In [ ]:
### 統合サンプルのヘテロ接合度を準備する

### サンプル名
samples = ["CEL_W", "FUK_W", "NIS_W", "KAT_W", "KUR_P", "FUK_P", "ONO_P", "KAT_P"]

import itertools
combinations = itertools.combinations(samples, 2)

for c in combinations:
  ## アリル数
  totalN = df[c[0]+"_A1"] + df[c[0]+"_A2"] + df[c[1]+"_A1"] + df[c[1]+"_A2"]
  N_A1 = df[c[0]+"_A1"] + df[c[1]+"_A1"]
  ## アリル頻度
  freqA1 = N_A1 / totalN
  freqA2 = 1 - freqA1
  ## ヘテロ接合度
  df[c[0]+"_"+c[1]+"_het"] = 2 * freqA1 * freqA2

### 各統合サンプルのヘテロ接合度データ
two = df.iloc[:, -28:].mean()
two

## [ 例 ] CEL_W-FUK_W間の遺伝的分化度 Fst

　サンプル間の遺伝的分化（Fst）の求め方を再掲すると、

- Fst = 1 - (H_1 + H_2)/2 / H_t
  - H_1: サンプル1のヘテロ接合度
  - H_2: サンプル2のヘテロ接合度
  - H_t: 統合サンプルのヘテロ接合度

（例） CEL_WとFUK_Wの遺伝的分化度 Fst
- CEL_W_het = 0.386984
- FUK_W_het = 0.384626
- CEL_W_FUK_W_het = 0.391060 
- Fst = 1 - (CEL_W_het + FUK_W_het)/2 / CEL_W_FUK_W_het 

In [ ]:
Fst = {}
Fst["CEL_W_FUK_W"] = 1 - (one["CEL_W_het"] + one["FUK_W_het"])/2 / two["CEL_W_FUK_W_het"]

## 表示
Fst["CEL_W_FUK_W"]

## [ 練習 ] CEL_W-NIS_W間の遺伝的分化度 Fst

In [ ]:
Fst["CEL_W_NIS_W"] = 

## 表示
Fst["CEL_W_NIS_W"]

## 全サンプルペアの遺伝的分化度 Fst

In [ ]:
### サンプル名
samples = ["CEL_W", "FUK_W", "NIS_W", "KAT_W", "KUR_P", "FUK_P", "ONO_P", "KAT_P"]

import itertools
combinations = itertools.combinations(samples, 2)

for c in combinations:
  ## ペアFst
  Fst[c[0]+"_"+c[1]] = 1 - (one[c[0]+"_het"] + one[c[1]+"_het"])/2 / two[c[0]+"_"+c[1]+"_het"]

## 表示
Fst

## 調べたFstをグラフに描く

In [ ]:
## グラフ用データ
### サンプル名
samples = ["CEL_W", "FUK_W", "NIS_W", "KAT_W", "KUR_P", "FUK_P", "ONO_P", "KAT_P"]

## 総当たりでペアFstを求めて、行列データにする
fst_matrics = []
for i in range(8):
  for j in range(8):
    ## ペアFst
    if i == j:
      fst = 1
    elif i < j:
      fst = 1 - (one[samples[i]+"_het"] + one[samples[j]+"_het"])/2 / two[samples[i]+"_"+samples[j]+"_het"]
    else:
      fst = 1 - (one[samples[j]+"_het"] + one[samples[i]+"_het"])/2 / two[samples[j]+"_"+samples[i]+"_het"]
    ## ペアFstをリストに追加
    fst_matrics.append(fst)

## 行列データ化
import numpy as np
fst_matrics = np.array(fst_matrics).reshape(8, 8)

## 表示
fst_matrics

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## Fstのヒートマップ
sns.heatmap(fst_matrics, cmap='coolwarm', 
            annot=True, mask=np.triu(fst_matrics),
            vmin=0.0, vmax=0.5,
            xticklabels=samples[:-1],
            yticklabels=[""]+samples[1:],)
plt.tick_params(bottom=False, left=False)

---
# 発表でまとめることの例
- 研究の目的や背景
  - 研究したサンプルについての基本情報
  - すでに知られていること、知られていないこと
  - 研究の目的（何を明らかにするか）
- 研究の手法
  - SNPとは何か？
  - 遺伝的分化度 Fst とは何か？
  - どうやってFstを求めたのか？
- 結果と考察
  - 調べた遺伝子数やSNP数の情報
  - どの程度サンプル間は遺伝的に分化していたか？



# 検討事項
- 上では、2つのサンプルペアの遺伝的多様性を調べた。
  - サンプルペアの地理的距離が大きいほど、遺伝的に分化しているのか？
  - 白花ニワゼキショウと紫花ニワゼキショウの間のFstはどのように調べればよいか？
  - 白花ニワゼキショウと紫花ニワゼキショウは遺伝的に分化しているのか？

- 今回得られた遺伝的分化度は大きいのか？それとも小さいのか？　どうすれば評価できるか？